In [1]:
# Imports
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning)

import os
from dotenv import load_dotenv
# Load environment variables from .env
load_dotenv()
MODEL_NAME = 'gpt-4o'
os.environ["OPENAI_MODEL_NAME"] = MODEL_NAME

# CrewAI tools
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool, FileReadTool
from PyPDF2 import PdfReader # allows you to extract text from PDFs
from langchain.chat_models import ChatOpenAI
from langchain.tools import Tool

# For PDF generation
from fpdf import FPDF # library to create PDF files

/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/pydantic/_internal/_config.py:323: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  @validator("website_url")
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: Pyd

## File Path

In [2]:
# Option A: Using the original function that reads from a folder
#folder_path = "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers"
#local_summaries = read_and_summarize_papers(folder_path)

# Option B: Use the manual file paths 
file_paths = [
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Adaptive Reasoning Language Agents.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Agents in Clinic.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Autonomous Agents 2024 in medicine.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/LLM Agents in Medicine.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/MedAide.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Multimodal in healthcare.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Polaris LLM Constellation.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Systematic Review LLM Apps.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/Transformative impact of LLM in Medicine.pdf",
    "/Users/brunamedeiros/Documents/University of Chicago/Spring 2025 - Capstone I/Assignment-Research (2)/Papers/yang-et-al-2024-application-of-large-language-models-in-disease-diagnosis-and-treatment.pdf"
    ]

## Helper Functions

In [3]:
# ==================
# PURPOSE: split long string (text) into smaller pieces with up to 15000 words each. This makes it easier to send smaller texts to the language model, avoiding size limits.
# ==================
def chunk_text(text, max_words=1500):
    """
    Splits the given text into smaller chunks, each containing at most max_words.
    This is useful when sending long text to the language model.
    """
    words = text.split()  # Split the text into a list of words.
    # Create chunks by joining groups of words back into strings.
    return [" ".join(words[i:i+max_words]) for i in range(0, len(words), max_words)]

# ==================
# PURPOSE: Reads a PDF file and extracts all text from it. The function loops through each page, extracts the text, and adds it to a single string.
# ==================
def extract_text_from_pdf(file_path):
    """
    Reads a PDF file and extracts text from each page.
    Returns the full text as a single string.
    """
    reader = PdfReader(file_path)  # Open the PDF file.
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()  # Get text from the current page.
        if page_text:  # Check if text is not None.
            text += page_text + "\n"  # Append page text with a newline.
    return text

# ==================
# PURPOSE: Reads each file from a given list of file paths and creates a summary.
# ==================
def read_and_summarize_papers_manual(agent, file_paths):
    """
    Reads and summarizes each paper specified in the list of file_paths.
    For each file:
      1. Reads the content (from PDF or text file).
      2. Splits the content into manageable chunks.
      3. Uses a language model (llm) to summarize each chunk with a structured prompt.
      4. Combines these partial summaries into one final structured summary.
      
    Returns a dictionary with filenames as keys and their summaries as values.
    """
    summaries = {}
    llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)

    for file_path in file_paths:
        filename = os.path.basename(file_path)
        print(f"\n🔍 Reading: {filename}")

        # Read the content from the file.
        if filename.endswith(".pdf"):
            content = extract_text_from_pdf(file_path)
        else:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()

        # Split content into chunks.
        chunks = chunk_text(content, max_words=1500)
        partial_summaries = []

        # Summarize each chunk.
        for i, chunk in enumerate(chunks):
            prompt = f"""
You are summarizing part {i+1} of an academic paper. Follow this structure exactly and do NOT make assumptions.

1. **AI Technique(s)**: What GenAI method(s) are used? (Name models, frameworks, architectures.)
2. **Healthcare Application**: What specific medical domain or task is this applied to?
3. **Methodology**: Detailed description of the model pipeline or process.
4. **Key Findings or Contributions**: What are the main results or insights?
5. **Limitations or Challenges**: Any weaknesses or barriers mentioned?

Text from the paper:
---------------------
{chunk}
"""
            summary = llm.predict(prompt)
            partial_summaries.append(summary)

        # Combine partial summaries into a final structured summary.
        combined_prompt = f"""
You are generating a final structured summary for the paper '{filename}'. Use only the information from the following parts. Do not add anything new. Follow this format exactly:

### {filename}
- **AI Technique(s)**:
- **Healthcare Application**:
- **Methodology**:
- **Key Findings or Contributions**:
- **Limitations or Challenges**:

Paper Parts:
---------------------
{chr(10).join(partial_summaries)}
"""
        final_summary = llm.predict(combined_prompt)
        summaries[filename] = final_summary
        print(f"✅ Done summarizing: {filename}\n")

    return summaries

## Agent Definitions

In [4]:
# Agent 1
local_reviewer_agent = Agent(
    role="Local Literature Reviewer & Gap Identifier",
    goal=(
        "Read academic papers on GenAI and Healthcare from a local folder and extract detailed research gaps and innovative ideas. "
        "Focus on advanced technical models (e.g., complex transformer architectures, attention mechanisms, diffusion models, GANs) and explain their clinical applicability."
    ),
    backstory=(
        "I am a seasoned research analyst specializing in biomedical literature and advanced GenAI methods. "
        "My expertise enables me to extract actionable insights, pinpoint technical shortcomings in current studies, and identify clinically relevant research gaps."
    ),
    tools=[],
    allow_delegation=False,
    memory=True,
    verbose=True
)

# Agent 2
web_search_tool = SerperDevTool()
web_researcher_agent = Agent(
    role="Web Literature Researcher & Idea Enhancer",
    goal=(
        "Search for recent academic papers and studies on GenAI in Healthcare from the web—specifically using PubMed as the primary source—and extract additional research gaps and innovative ideas. "
        "Search for them specifically at 'https://pubmed.ncbi.nlm.nih.gov' and produce a short (3-sentence max) summary of the most relevant findings."
        "Emphasize advanced generative AI models and ensure proposals are technically detailed and clinically actionable."
    ),
    backstory=(
        "I am an expert in web-based academic research. I use online search tools (targeting PubMed) to retrieve and synthesize the latest breakthroughs and practical insights in GenAI for Healthcare."
    ),
    tools=[web_search_tool],
    allow_delegation=False,
    memory=True,
    verbose=False
)

# Agent 3
critic_agent = Agent(
    role="Research Critique & Decision Maker",
    goal=(
        "Critically evaluate the research ideas generated from both local and web analyses. Score each idea on originality, feasibility, technical depth, and clinical impact. "
        "Select the most promising and innovative research direction."
    ),
    backstory=(
        "With extensive experience as a medical data scientist and healthcare strategist, I evaluate research ideas to ensure they are both innovative and directly translatable to clinical practice."
    ),
    allow_delegation=True,
    memory=True,
    verbose=True
)

# Agent 4
writer_agent = Agent(
    role="Research Proposal Writer",
    goal=(
        "Compose a comprehensive and detailed research proposal based on the selected research idea. "
        "Include sections for Title, Abstract, Background & Literature Review, Problem Statement & Research Gap, Proposed GenAI Approach, Expected Impact in Healthcare, Limitations or Ethical Considerations, and References."
    ),
    backstory=(
        "I specialize in transforming complex research ideas into well-structured, in-depth proposals that are both technically robust and clinically relevant. "
        "My writing ensures that every section is detailed and evidence-based."
    ),
    allow_delegation=False,
    memory=True,
    verbose=True
)

In [5]:
# Run summarization and save to file
local_summaries = read_and_summarize_papers_manual(local_reviewer_agent, file_paths)

with open("summaries_output.txt", "w", encoding="utf-8") as f:
    for filename, summary in local_summaries.items():
        f.write(f"### {filename}\n{summary}\n\n")

# Load summaries for use in Task input
with open("summaries_output.txt", "r", encoding="utf-8") as f:
    summaries_text = f.read()

/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_89477/116180402.py:44: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name=MODEL_NAME, temperature=0)
/var/folders/qf/l8sfrcpd6vq2564v1_qt31fm0000gn/T/ipykernel_89477/116180402.py:76: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = llm.predict(prompt)



🔍 Reading: Adaptive Reasoning Language Agents.pdf


/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic

✅ Done summarizing: Adaptive Reasoning Language Agents.pdf


🔍 Reading: Agents in Clinic.pdf


/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic

...

/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic

✅ Done summarizing: Transformative impact of LLM in Medicine.pdf


🔍 Reading: yang-et-al-2024-application-of-large-language-models-in-disease-diagnosis-and-treatment.pdf


/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()
/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic

✅ Done summarizing: yang-et-al-2024-application-of-large-language-models-in-disease-diagnosis-and-treatment.pdf



/opt/anaconda3/envs/capstone/lib/python3.10/site-packages/langchain_community/chat_models/openai.py:495: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  response = response.dict()


## Task Definitions

In [7]:
# Task 1: Local Literature Analysis & Research Gap Identification
local_analysis_task = Task(
    description=(
        "Read academic papers from the local folder on GenAI and Healthcare and extract detailed research gaps and innovative ideas. "
        "Include technical details on advanced generative AI models and explain their clinical relevance. This should be no more than 3-setences long."
        f"PAPER SUMMARIES:\n{summaries_text}"
    ),
    expected_output=(
        "3-sentence summary per paper. No extra text."
    ),
    agent=local_reviewer_agent,
    input=f"Analyze these local paper summaries:\n{summaries_text}"
)

# Task 2: Web Literature Research & Idea Augmentation
web_analysis_task = Task(
    description=(
        "Search for recent online academic papers on GenAI in Healthcare and extract additional research gaps and innovative ideas. "
        "Emphasize advanced generative AI models and actionable clinical applications."
        "Use the web search tool on 'https://pubmed.ncbi.nlm.nih.gov' with query 'Generative AI in Healthcare' and produce a single 3-sentence summary of the most relevant recent insights or findings."
    ),
    expected_output=(
        "3-sentence summary per paper. No extra text."
    ),
    agent=web_researcher_agent
)

# Task 3: Research Critique, Evaluation & Decision
critique_task = Task(
    description=(
        "Review the short local and web summaries. Score the identified ideas (1–5) on originality, feasibility, tech depth, and impact. "
        "Select the best idea in no more than 3 sentences total."
    ),
    expected_output=(
        "The top 5 best ideas with short justification (max 3 sentences)."
        "Finally, specify which one is the final picked idea, and explain why this is the best one."
    ),
    agent=critic_agent
)

# Task 4: Comprehensive Research Proposal Generation
proposal_task = Task(
    description=(
        "Write a complete research proposal based on the selected research idea. "
        "The proposal must include the following sections: Title, Abstract (150–250 words), Background & Literature Review (with detailed discussions of previous studies, authors, models, innovations, and results), "
        "Problem Statement & Research Gap (with concrete evidence), Proposed GenAI Approach (specifying advanced models and techniques, e.g., transformer architectures, diffusion models, GANs), "
        "Expected Impact in Healthcare, Limitations or Ethical Considerations, and References."
    ),
    expected_output="A comprehensive, technically rigorous, and clinically applicable research proposal document.",
    agent=writer_agent
)

## Crew Setup & Workflow Execution

In [ ]:
crew = Crew(
    agents=[local_reviewer_agent, web_researcher_agent, critic_agent, writer_agent],
    tasks=[local_analysis_task, web_analysis_task, critique_task, proposal_task],
    verbose=False
)

if __name__ == "__main__":
    print("\n🚀 Running multi-agent research proposal workflow...")
    result = crew.kickoff()
    print("\n✅ Final Proposal:\n")
    print(result)

    # Save plain text version
    with open("final_proposal.txt", "w", encoding="utf-8") as f:
        f.write(str(result))
    print("\n📄 Proposal saved to final_proposal.txt")

    # Save formatted PDF
    from fpdf import FPDF
    from datetime import datetime

    import unicodedata
    def clean_text(text):
        """Convert fancy unicode to plain ASCII where possible"""
        return unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii')
    
    class StyledPDF(FPDF):
        def header(self):
            if self.page_no() > 1:
                self.set_font("Times", "B", 12)
                self.set_text_color(0, 0, 100)
                self.cell(0, 10, "GenAI in Healthcare Research Proposal", ln=True, align="C")
                self.ln(5)

        def title_page(self, title):
            self.set_font("Times", "B", 20)
            self.set_text_color(0, 0, 100)
            self.cell(0, 60, "", ln=True)  # vertical spacing
            self.cell(0, 10, title, ln=True, align="C")
            self.set_font("Times", "", 14)
            self.ln(10)
            self.cell(0, 10, "Generated by CrewAI Agents", ln=True, align="C")
            self.cell(0, 10, datetime.now().strftime("Date: %B %d, %Y"), ln=True, align="C")
            self.add_page()

        def add_section(self, title, body):
            self.set_font("Times", "B", 14)
            self.set_text_color(0, 0, 128)
            self.cell(0, 10, title, ln=True)
            self.ln(2)
            self.set_font("Times", "", 12)`
            self.set_text_color(0, 0, 0)
            for paragraph in body.strip().split("\n\n"):
                self.multi_cell(0, 10, paragraph.strip())
                self.ln(1)

    # Create and build PDF
    pdf = StyledPDF()
    pdf.add_page()
    pdf.title_page("GenAI in Healthcare Re`search Proposal")

    # Parse by section headings (split on markdown-style titles)
    sections = str(result).split("**")
    for i in range(1, len(sections), 2):  # step over title-body pairs
        section_title = clean_text(sections[i].strip(":").strip())
        section_body = clean_text(sections[i+1].strip()) if i+1 < len(sections) else ""
        pdf.add_section(section_title, section_body)

    pdf.output("final_proposal.pdf")
    print("\n📄 Styled proposal PDF saved to final_proposal.pdf")



🚀 Running multi-agent research proposal workflow...
# Agent: Local Literature Reviewer & Gap Identifier
## Task: Read academic papers from the local folder on GenAI and Healthcare and extract detailed research gaps and innovative ideas. Include technical details on advanced generative AI models and explain their clinical relevance. This should be no more than 3-setences long.PAPER SUMMARIES:
### Adaptive Reasoning Language Agents.pdf
### Adaptive Reasoning Language Agents.pdf
- **AI Technique(s)**: The paper utilizes large language models (LLMs), specifically GPT-4 and GPT-3.5, as the GenAI methods. These models are employed as language agents in the study.

- **Healthcare Application**: The application is in the domain of diagnostic accuracy within simulated clinical environments, specifically using the AgentClinic benchmark and the MedQA dataset.

- **Methodology**: The methodology involves an LLM agent framework where doctor agents iteratively refine their reasoning and actions fol

---